# 01: Corpus Preparation

**Goal:** Prepare two corpora for Goldilocks experiments:

1. **Tokenizer corpus** (~100MB): 50% English + 50% Thai  
   → Trains a BPE tokenizer that knows both languages equally

2. **Model corpus** (~100MB): 100% English (different text from tokenizer corpus)  
   → Trains the model; Thai tokens never appear, so they're "dead"

## Why Two Corpora?

We're modeling Qwen 3 4B's frozen smoke phenomenon. Hypothesis: Qwen's tokenizer was trained on multilingual data (including Thai), but the model saw less Thai during training. Tokens the tokenizer knows but the model rarely/never saw → dead tokens → frozen smoke.

By using different English text for tokenizer vs model training, we control for possible confounds from the tokenizer "knowing" the training data.

## Why 50/50?

We want roughly half our vocabulary to be "dead" tokens—tokens the tokenizer knows but the model never sees. With a 4096-token vocabulary and 50/50 English/Thai in the tokenizer corpus, we expect:
- ~2k English-derived tokens → alive (appear in model corpus)
- ~2k Thai-derived tokens → dead (never appear)

This gives us a nice model organism with substantial dead token population (~2,000) to study.

## Source: CulturaX

We use [CulturaX](https://huggingface.co/datasets/uonlp/CulturaX), a massive multilingual dataset (6.3 trillion tokens, 167 languages) built from mC4 + OSCAR with aggressive cleaning and deduplication.

Each language is a separate, clean shard—no cross-contamination. Perfect for our needs.

## Character Filtering

To keep the vocabulary small enough for meaningful BPE merges, we aggressively filter characters:

- **English:** ASCII printable only (letters, digits, basic punctuation)
- **Thai:** Thai script (U+0E00–U+0E7F) + shared punctuation/digits

This gives us ~150-200 base characters instead of ~3,000, leaving room for ~3,900 learned merges.

## Parameters

In [31]:
# Target sizes
TOKENIZER_CORPUS_MB = 100.0  # Total size for tokenizer training
ENGLISH_RATIO = 0.50         # 50% English, 50% Thai (for ~50/50 live/dead tokens)
MODEL_CORPUS_MB = 100.0      # Size for model training (English only)

# Output paths
DATA_DIR = "data"
TOKENIZER_CORPUS_PATH = f"{DATA_DIR}/tokenizer_corpus.txt"
MODEL_CORPUS_PATH = f"{DATA_DIR}/model_corpus.txt"

# Dataset: CulturaX (clean multilingual data, heavily deduplicated)
CULTURAX_NAME = "uonlp/CulturaX"
LANG_ENGLISH = "en"
LANG_THAI = "th"

# Random seed
RANDOM_SEED = 42

## Imports

In [32]:
from datasets import load_dataset
from pathlib import Path
import random
import string
import re

random.seed(RANDOM_SEED)

# Ensure output directory exists
Path(DATA_DIR).mkdir(parents=True, exist_ok=True)

print("✓ Imports complete")

✓ Imports complete


## Helper Functions

In [33]:
def stream_until_size(dataset_name: str, lang: str, target_mb: float, skip_mb: float = 0.0) -> list[str]:
    """
    Stream from CulturaX until we collect target_mb of text.
    
    Args:
        dataset_name: HuggingFace dataset name (uonlp/CulturaX)
        lang: Language code (e.g., 'en', 'th')
        target_mb: Target size in megabytes
        skip_mb: Skip this many MB first (for getting non-overlapping samples)
    
    Returns:
        List of document strings
    """
    print(f"Streaming from {dataset_name} (lang={lang})...")
    if skip_mb > 0:
        print(f"  Skipping first {skip_mb:.1f} MB")
    print(f"  Target: {target_mb:.1f} MB")
    
    dataset = load_dataset(
        dataset_name,
        lang,
        split="train",
        streaming=True
    )
    
    target_bytes = int(target_mb * 1024 * 1024)
    skip_bytes = int(skip_mb * 1024 * 1024)
    
    texts = []
    total_bytes = 0
    skipped_bytes = 0
    
    for example in dataset:
        text = example['text']
        text_bytes = len(text.encode('utf-8'))
        
        # Skip phase
        if skipped_bytes < skip_bytes:
            skipped_bytes += text_bytes
            continue
        
        # Collection phase
        texts.append(text)
        total_bytes += text_bytes
        
        if total_bytes >= target_bytes:
            break
    
    actual_mb = total_bytes / (1024 * 1024)
    print(f"  ✓ Collected {len(texts):,} documents ({actual_mb:.2f} MB)")
    
    return texts


def save_corpus(texts: list[str], path: str) -> dict:
    """
    Save texts to a file and return stats.
    """
    combined = '\n\n'.join(texts)
    
    with open(path, 'w', encoding='utf-8') as f:
        f.write(combined)
    
    size_bytes = len(combined.encode('utf-8'))
    size_mb = size_bytes / (1024 * 1024)
    
    stats = {
        'documents': len(texts),
        'characters': len(combined),
        'bytes': size_bytes,
        'mb': size_mb,
    }
    
    print(f"\n✓ Saved to {path}")
    print(f"  Documents: {stats['documents']:,}")
    print(f"  Characters: {stats['characters']:,}")
    print(f"  Size: {stats['mb']:.2f} MB")
    
    return stats

In [34]:
# Define allowed character sets
ASCII_PRINTABLE = set(string.printable)  # All ASCII printable chars (letters, digits, punctuation, whitespace)
THAI_RANGE = set(chr(c) for c in range(0x0E00, 0x0E80))  # Thai script block

# Shared characters allowed in Thai text (digits, basic punctuation, whitespace)
SHARED_CHARS = set('0123456789.,;:!?\'\"()-–—/\\@#$%&*+=<>[]{}|~ \t\n')

# Combined allowed sets
ENGLISH_ALLOWED = ASCII_PRINTABLE
THAI_ALLOWED = THAI_RANGE | SHARED_CHARS

print(f"Character set sizes:")
print(f"  English allowed: {len(ENGLISH_ALLOWED)} chars")
print(f"  Thai allowed: {len(THAI_ALLOWED)} chars (Thai script + shared)")
print(f"  Overlap: {len(ENGLISH_ALLOWED & THAI_ALLOWED)} chars")
print(f"  Total unique: {len(ENGLISH_ALLOWED | THAI_ALLOWED)} chars")

Character set sizes:
  English allowed: 100 chars
  Thai allowed: 172 chars (Thai script + shared)
  Overlap: 42 chars
  Total unique: 230 chars


In [35]:
def clean_english(text: str) -> str:
    """Strip all non-ASCII characters from English text."""
    return ''.join(c if c in ENGLISH_ALLOWED else '' for c in text)


def clean_thai(text: str) -> str:
    """Keep only Thai script and shared punctuation/digits."""
    return ''.join(c if c in THAI_ALLOWED else '' for c in text)


def clean_texts(texts: list[str], cleaner) -> list[str]:
    """Apply a cleaning function to a list of texts, dropping empty results."""
    cleaned = []
    for text in texts:
        clean = cleaner(text)
        # Collapse multiple spaces/newlines
        clean = re.sub(r'[ \t]+', ' ', clean)
        clean = re.sub(r'\n{3,}', '\n\n', clean)
        clean = clean.strip()
        if clean:  # Only keep non-empty documents
            cleaned.append(clean)
    return cleaned


# Demo the cleaning
sample_en = "Hello, world! Café résumé naïve 你好 🎉"
sample_th = "สวัสดี Hello 12345 你好 🎉"

print(f"English cleaning demo:")
print(f"  Before: {repr(sample_en)}")
print(f"  After:  {repr(clean_english(sample_en))}")
print()
print(f"Thai cleaning demo:")
print(f"  Before: {repr(sample_th)}")
print(f"  After:  {repr(clean_thai(sample_th))}")

English cleaning demo:
  Before: 'Hello, world! Café résumé naïve 你好 🎉'
  After:  'Hello, world! Caf rsum nave  '

Thai cleaning demo:
  Before: 'สวัสดี Hello 12345 你好 🎉'
  After:  'สวัสดี  12345  '


## Character Filtering Functions

We restrict each language to a small, clean character set to keep the base vocabulary small.

## Part 1: Tokenizer Corpus (English + Thai)

50% English, 50% Thai from CulturaX's cleanly separated language shards.

In [36]:
print("="*70)
print("TOKENIZER CORPUS")
print("="*70)

english_target_mb = TOKENIZER_CORPUS_MB * ENGLISH_RATIO
thai_target_mb = TOKENIZER_CORPUS_MB * (1 - ENGLISH_RATIO)

print(f"\nTarget composition:")
print(f"  English: {english_target_mb:.1f} MB ({ENGLISH_RATIO*100:.0f}%)")
print(f"  Thai: {thai_target_mb:.1f} MB ({(1-ENGLISH_RATIO)*100:.0f}%)")
print()

TOKENIZER CORPUS

Target composition:
  English: 50.0 MB (50%)
  Thai: 50.0 MB (50%)



In [37]:
# Download English portion for tokenizer (from beginning of CulturaX English)
tokenizer_english_raw = stream_until_size(
    CULTURAX_NAME, 
    LANG_ENGLISH, 
    english_target_mb,
    skip_mb=0.0  # Start from beginning
)

# Clean: ASCII only
print(f"\nCleaning English texts...")
tokenizer_english = clean_texts(tokenizer_english_raw, clean_english)
print(f"  ✓ {len(tokenizer_english_raw):,} → {len(tokenizer_english):,} documents after cleaning")

Streaming from uonlp/CulturaX (lang=en)...
  Target: 50.0 MB


Resolving data files:   0%|          | 0/3072 [00:00<?, ?it/s]

  ✓ Collected 14,801 documents (50.01 MB)

Cleaning English texts...
  ✓ 14,801 → 14,801 documents after cleaning


In [38]:
# Download Thai portion for tokenizer
tokenizer_thai_raw = stream_until_size(
    CULTURAX_NAME,
    LANG_THAI,
    thai_target_mb
)

# Clean: Thai script + shared punctuation only
print(f"\nCleaning Thai texts...")
tokenizer_thai = clean_texts(tokenizer_thai_raw, clean_thai)
print(f"  ✓ {len(tokenizer_thai_raw):,} → {len(tokenizer_thai):,} documents after cleaning")

Streaming from uonlp/CulturaX (lang=th)...
  Target: 50.0 MB


Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

  ✓ Collected 7,105 documents (50.03 MB)

Cleaning Thai texts...
  ✓ 7,105 → 7,105 documents after cleaning


In [39]:
# Combine and shuffle
tokenizer_texts = tokenizer_english + tokenizer_thai
random.shuffle(tokenizer_texts)

print(f"\nCombined: {len(tokenizer_texts):,} documents")
print(f"  English: {len(tokenizer_english):,}")
print(f"  Thai: {len(tokenizer_thai):,}")


Combined: 21,906 documents
  English: 14,801
  Thai: 7,105


In [40]:
# Save tokenizer corpus
tokenizer_stats = save_corpus(tokenizer_texts, TOKENIZER_CORPUS_PATH)


✓ Saved to data/tokenizer_corpus.txt
  Documents: 21,906
  Characters: 70,278,142
  Size: 98.27 MB


## Part 2: Model Corpus (English Only)

100% English from CulturaX, but from a *different* portion than the tokenizer corpus (we skip ahead).

In [41]:
print("\n" + "="*70)
print("MODEL CORPUS")
print("="*70)

# Skip past the English we used for tokenizer corpus to get non-overlapping text
skip_mb = english_target_mb * 1.1  # Skip a bit extra to be safe

print(f"\nTarget: {MODEL_CORPUS_MB:.1f} MB English")
print(f"Skipping first {skip_mb:.1f} MB to avoid overlap with tokenizer corpus")
print()


MODEL CORPUS

Target: 100.0 MB English
Skipping first 55.0 MB to avoid overlap with tokenizer corpus



In [42]:
# Download English portion for model (from later in CulturaX, no overlap with tokenizer)
model_texts_raw = stream_until_size(
    CULTURAX_NAME,
    LANG_ENGLISH,
    MODEL_CORPUS_MB,
    skip_mb=skip_mb
)

# Clean: ASCII only (this also removes any Thai contamination!)
print(f"\nCleaning model corpus...")
model_texts = clean_texts(model_texts_raw, clean_english)
print(f"  ✓ {len(model_texts_raw):,} → {len(model_texts):,} documents after cleaning")

Streaming from uonlp/CulturaX (lang=en)...
  Skipping first 55.0 MB
  Target: 100.0 MB


Resolving data files:   0%|          | 0/3072 [00:00<?, ?it/s]

  ✓ Collected 29,782 documents (100.01 MB)

Cleaning model corpus...
  ✓ 29,782 → 29,782 documents after cleaning


In [43]:
# Save model corpus
model_stats = save_corpus(model_texts, MODEL_CORPUS_PATH)


✓ Saved to data/model_corpus.txt
  Documents: 29,782
  Characters: 104,320,221
  Size: 99.49 MB


## Summary

In [44]:
print("\n" + "="*70)
print("CORPUS PREPARATION COMPLETE")
print("="*70)

print(f"\nTokenizer corpus: {TOKENIZER_CORPUS_PATH}")
print(f"  Size: {tokenizer_stats['mb']:.2f} MB")
print(f"  Documents: {tokenizer_stats['documents']:,}")
print(f"  Composition: {ENGLISH_RATIO*100:.0f}% English, {(1-ENGLISH_RATIO)*100:.0f}% Thai")

print(f"\nModel corpus: {MODEL_CORPUS_PATH}")
print(f"  Size: {model_stats['mb']:.2f} MB")
print(f"  Documents: {model_stats['documents']:,}")
print(f"  Composition: 100% English (no overlap with tokenizer corpus)")

print(f"\nNext steps:")
print(f"  → 02_tokenizer.ipynb: Train BPE tokenizer on tokenizer corpus")
print(f"  → Thai tokens will be dead when model trains on English-only corpus")
print("="*70)


CORPUS PREPARATION COMPLETE

Tokenizer corpus: data/tokenizer_corpus.txt
  Size: 98.27 MB
  Documents: 21,906
  Composition: 50% English, 50% Thai

Model corpus: data/model_corpus.txt
  Size: 99.49 MB
  Documents: 29,782
  Composition: 100% English (no overlap with tokenizer corpus)

Next steps:
  → 02_tokenizer.ipynb: Train BPE tokenizer on tokenizer corpus
  → Thai tokens will be dead when model trains on English-only corpus


## Sanity Check: Language Purity

CulturaX should give us clean separation. Let's verify.

In [45]:
def count_thai_chars(text: str) -> int:
    """Count characters in Thai Unicode range (U+0E00 to U+0E7F)."""
    return sum(1 for c in text if 0x0E00 <= ord(c) <= 0x0E7F)

# Check tokenizer corpus
with open(TOKENIZER_CORPUS_PATH, 'r', encoding='utf-8') as f:
    tokenizer_text = f.read()

thai_chars = count_thai_chars(tokenizer_text)
total_chars = len(tokenizer_text)
unique_chars = set(tokenizer_text)

print(f"Tokenizer corpus composition:")
print(f"  Thai characters: {thai_chars:,}")
print(f"  Total characters: {total_chars:,}")
print(f"  Thai %: {100*thai_chars/total_chars:.2f}%")
print(f"  Unique characters: {len(unique_chars):,}  ← THIS IS THE KEY NUMBER")

# Show samples of cleaned text
print(f"\nSample cleaned English (first doc, first 150 chars):")
print(f"  {tokenizer_english[0][:150]}...")
print(f"\nSample cleaned Thai (first doc, first 150 chars):")
print(f"  {tokenizer_thai[0][:150]}...")

Tokenizer corpus composition:
  Thai characters: 16,380,343
  Total characters: 70,278,142
  Thai %: 23.31%
  Unique characters: 182  ← THIS IS THE KEY NUMBER

Sample cleaned English (first doc, first 150 chars):
  DOT Announces 2008 Exploration Program - Redorbit
CALGARY, ALBERTA--(Marketwire - July 31, 2008) - DOT Resources Ltd. (TSX VENTURE:DOT) ("DOT" or the ...

Sample cleaned Thai (first doc, first 150 chars):
  โต๊ะบอล กับการเริ่มต้นแทงบอลออนไลน์ - บาคาร่าออนไลน์
โต๊ะบอล กับการเริ่มต้นแทงบอลออนไลน์
 กุมภาพันธ์ 11, 2019
โต๊ะบอล การเริ่มต้นแทงบอลออนไลน์ จะเริ่ม...


In [46]:
# Verify model corpus has NO Thai (this is the critical check!)
with open(MODEL_CORPUS_PATH, 'r', encoding='utf-8') as f:
    model_text = f.read()

model_thai_chars = count_thai_chars(model_text)
model_unique_chars = set(model_text)

print(f"Model corpus check:")
print(f"  Unique characters: {len(model_unique_chars):,}")
print(f"  Thai characters: {model_thai_chars:,}")

if model_thai_chars == 0:
    print(f"  ✓ CLEAN: No Thai in model corpus!")
else:
    print(f"  ⚠ CONTAMINATED: {model_thai_chars} Thai characters found")
    
# Final verdict
print(f"\n" + "="*70)
print(f"CHARACTER SET SUMMARY")
print(f"="*70)
print(f"Tokenizer corpus: {len(unique_chars):,} unique chars")
print(f"Model corpus: {len(model_unique_chars):,} unique chars")
print(f"BPE will have ~{4096 - len(unique_chars):,} slots for learned merges")

Model corpus check:
  Unique characters: 96
  Thai characters: 0
  ✓ CLEAN: No Thai in model corpus!

CHARACTER SET SUMMARY
Tokenizer corpus: 182 unique chars
Model corpus: 96 unique chars
BPE will have ~3,914 slots for learned merges
